In [ ]:
# default_exp tftools

# tftools

> Functions for use with TensorFlow 2.x

## References  
The functions below are based on the below sources. Please visit attached links for further understanding.:
- Chapter 14 – Deep Computer Vision Using Convolutional Neural Networks, Hands-on Machine Learning with Scikit-Learn, Keras and TensorFlow 2nd Edition by Aurélien Geron: [Github link to notebook](https://github.com/ageron/handson-ml2/blob/master/14_deep_computer_vision_with_cnns.ipynb)
    - In particular, functions `random_crop` and `central_crop` are used
- Tensorflow Documentation by The Tensorflow Authors:
    - [Load Images Notebook Github Link](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/load_data/images.ipynb)
    - [tf.image API documentation](https://www.tensorflow.org/api_docs/python/tf/image)
    - [Transfer learning with a pretrained ConvNet](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/images/transfer_learning.ipynb)

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from functools import partial

## Image Augmentation

In [ ]:
#export
def random_crop(image):
    "Random crop of image"
    shape = tf.shape(image)
    min_dim = tf.reduce_min([shape[0], shape[1]]) * 90 // 100
    return tf.image.random_crop(image, [min_dim, min_dim, 3])

In [ ]:
#export
def central_crop(image):
    "Center crop of image"
    shape = tf.shape(image)
    min_dim = tf.reduce_min([shape[0], shape[1]])
    top_crop = (shape[0] - min_dim) // 4
    bottom_crop = shape[0] - top_crop
    left_crop = (shape[1] - min_dim) // 4
    right_crop = shape[1] - left_crop
    return image[top_crop:bottom_crop, left_crop:right_crop]

In [ ]:
#export
def random_flip(image, horiz=True, vert=False):
    "Randomly flips an image horizontally and/or vertically"
    img = image
    if horiz: img = tf.image.random_flip_left_right(img)
    if vert: img = tf.image.random_flip_up_down(img)
    return img

In [ ]:
#export
def random_brightness(image):
    "Randomly adjust brightness of image"
    return tf.image.random_brightness(image, max_delta = 0.3)

In [ ]:
#export
def random_contrast(image):
    "Randomly adjust contrast of image"
    return tf.image.random_contrast(image, 0, 0.3)

In [ ]:
#export
def random_rotate(image):
    """
    Randomly rotates image and angles are in radians.
    Requires tensorflow-addons that currently supports MacOS and Linux only.
    """
    import tensorflow as tf
    import tensorflow_addons as tfa
    delta = tf.random.uniform([], -0.5, 0.5)
    return tfa.image.rotate(image, delta)

## Dataset reading and processing

In [ ]:
#export
def get_label(file_path, CLASS_NAMES):
    "Return label of image for tf.data.Dataset"
    parts = tf.strings.split(file_path, os.path.sep)
    return parts[-2] == CLASS_NAMES

In [ ]:
#export
def train_test_split(files, valid_pct=0.2, seed=None):
    "Reads in list of file Path objects and randomly split into training and validation"
    files = sorted(files)
    if seed is not None:
        np.random.seed(seed)
    np.random.shuffle(files)
    cut = int((1-valid_pct) * len(files))
    return files[:cut], files[cut:]

In [ ]:
#export
def process_img_path(file_path, CLASS_NAMES=None, img_size=224, augments=None, mode=None):
    """
    process image for use with tf.data map function
    - get label
    - read and decode using tf.image
    - expects two augmentation function lists - one for train, the other for valid/test
    """
    label = get_label(file_path,CLASS_NAMES)
    img = tf.io.read_file(file_path)
    img = tf.image.decode_image(img, channels=3, expand_animations=False)
    img = tf.image.convert_image_dtype(img, tf.float32)
    if augments is not None:
        train_aug, valid_aug = augments
        if mode == 'train':
            for f in train_aug:
                img = f(img)
        else:
            for f in valid_aug:
                img = f(img)
        img = tf.clip_by_value(img, 0, 1)
    return tf.image.resize(img, [img_size, img_size]), label

In [ ]:
#export
def process_img_bytes(img_bytes, img_size=224, augments=None):
    """
    Process single image in int-byte form for use with keras model.predict()
    - read with in raw file bytes with functions like tf.io.read_file
    - decode using tf.image
    - supports one augmentation function list, typically for validation
    """
    try:
        img = tf.image.decode_image(img_bytes, channels=3, expand_animations=False)
        img = tf.image.convert_image_dtype(img, tf.float32)
        if augments is not None:
            for f in augments:
                img = f(img)
            img = tf.clip_by_value(img, 0, 1)
        img = tf.image.resize(img, [img_size, img_size])
    except Exception as e:
        print(f'{e}')
    return tf.expand_dims(img, 0)

In [ ]:
#export
def read_img_dataset(file_paths, CLASS_NAMES=None, shuffle_size=None, img_size=224, batch_size=32, n_parallel=4, augments=None, mode='train'):
    """
    Image dataset reader for tf.data.Dataset
    - get files from folder/list of Pathlib objects
    - modes of operation: train, valid, test
    - cache for all modes using local disk cache with mode as name of cache
    - only shuffle if mode=train. shuffle expects a shuffle size
    """
    ds = tf.data.Dataset.list_files(file_paths)
    ds = ds.map(partial(process_img_path,
                        CLASS_NAMES=CLASS_NAMES,
                        img_size=img_size,
                        augments=augments,
                        mode=mode), num_parallel_calls=n_parallel)
    ds = ds.cache(mode)
    if mode == 'train':
        ds = ds.shuffle(shuffle_size)
    ds = ds.repeat()
    ds = ds.batch(batch_size)
    return ds.prefetch(batch_size)

### Example: Read images from folder of Imagenet style
Sample:  
- data
    - train
        - class1
        - class2
    - valid
        - class1
        - class2

In [ ]:
# datapath = Path('path/to/data')
# train_aug = [rcrop, rflip]
# valid_aug = [crop]
# aug = (train_aug, valid_aug)
# train_ds = read_img_dataset(str(datapath/train/'*/*'), shuffle=1024, img_size=224, batch_size=32, n_parallel=4, augments=aug, mode='train')
# valid_ds = read_img_dataset(str(datapath/valid/'*/*'), img_size=224, batch_size=32, n_parallel=4, augments=aug, mode='valid')

### Example: Read images from folder (or list) and then shuffle split into train and validation

In [ ]:
# datapath = Path('path/to/data')
# all_files = get_all_files(datapath, recurse=True) # read from folder, can be list
# train_filepaths, tmp_filepaths = train_test_split(all_files, valid_pct=0.3, seed=42)
# valid_filepaths, test_filepaths = train_test_split(tmp_filepaths, valid_pct=0.5, seed=42)

In [ ]:
# train_ds = read_img_dataset([str(x) for x in train_filepaths], shuffle_size=1024, img_size=IMG_SIZE, batch_size=BATCH_SIZE, n_parallel=4, augments=aug, mode='train')
# valid_ds = read_img_dataset([str(x) for x in valid_filepaths], img_size=IMG_SIZE, batch_size=BATCH_SIZE, n_parallel=4, augments=aug, mode='valid')
# test_ds = read_img_dataset([str(x) for x in test_filepaths], img_size=IMG_SIZE, batch_size=BATCH_SIZE, n_parallel=4, augments=aug, mode='test')

## Visualizations

In [ ]:
#export
def show_batch(dataset, CLASS_NAMES):
    "displays batch of 25 images from a batch of tf.data.Dataset"
    image_batch, label_batch = next(iter(dataset))
    image_batch = image_batch.numpy()
    label_batch = label_batch.numpy()
    plt.figure(figsize=(10,10))
    for n in range(25):
        ax = plt.subplot(5,5,n+1)
        plt.imshow(image_batch[n])
        plt.title(CLASS_NAMES[label_batch[n]==1][0].title())
        plt.axis('off')

In [ ]:
#export
def plot_history(history):
    """
    Plots accuracy and loss for training and validation
    Needs history output from model.fit()
    """
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']

    loss = history.history['loss']
    val_loss = history.history['val_loss']

    plt.figure(figsize=(8, 8))
    plt.subplot(2, 1, 1)
    plt.plot(acc, label='Training Accuracy')
    plt.plot(val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.ylabel('Accuracy')
    plt.ylim([min(plt.ylim()),1])
    plt.title('Training and Validation Accuracy')

    plt.subplot(2, 1, 2)
    plt.plot(loss, label='Training Loss')
    plt.plot(val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.ylabel('Cross Entropy')
    plt.ylim([0,1.0])
    plt.title('Training and Validation Loss')
    plt.xlabel('epoch')
    plt.show()

- Plot history from model.fit using tf.keras

In [ ]:
# plot_history(history)

In [ ]:
#hide
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted 01_imgtools.ipynb.
Converted 02_tftools.ipynb.
Converted index.ipynb.
